In [7]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import AgglomerativeClustering
from transformers import AutoTokenizer, AutoModel
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer, WordNetLemmatizer
import string

In [20]:
df1 = pd.read_excel('231106_Labeling KI Files_x1121.xlsx')
df2 = pd.read_excel('2501 - 5000 final.xlsx')
df3 = pd.read_excel('1 - 2500 Fertig 1.xlsx')
df4 = pd.read_excel('230613_CSS Call Data Labeling.xlsx')

In [9]:
# Concatenate the DataFrames along the rows
df = pd.concat([df1, df2, df3], ignore_index=True)

df

,AP,Ariadne_internal_ID,content,cnt,VVL Mobile,VVL DSL,PO GA,I@H GA,TV GA,OOC (out of category),Comment,Conditional Formatting,Bearbeitungszeit
0,Katrin Kühne,FA163E153AE3-1b27-eeeaa700-77b26-64e3046f-1555...,Herzlich willkommen beim O Kundenservice mein ...,54,False,False,False,False,False,False,NaN,0,NaN
1,Katrin Kühne,FA163E153AE3-1b27-eeeaa700-77d03-64e3074a-24e4...,Guten Morgen bei O sie sprechen mit Alexandra ...,61,False,False,True,False,False,False,NaN,1,NaN
2,Katrin Kühne,FA163E153AE3-1b27-eeeaa700-77d06-64e3074a-6d7f...,Ackermann fertig willkommen bei O sie sprechen...,62,False,False,False,False,False,False,NaN,0,NaN
3,Katrin Kühne,FA163E153AE3-1b27-eeeaa700-77d0f-64e30753-8296...,Herzlich willkommen im O Service mein Name ist...,63,False,False,False,False,False,False,NaN,0,NaN
4,Katrin Kühne,FA163E153AE3-1b27-eeeaa700-77d1e-64e30771-785f...,Guten Morgen herzlich willkommen bei O sie spr...,64,False,False,True,False,False,False,NaN,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6117,Anja Haye-Heinemann,FA163E153AE3-1b27-efeac700-8560b-64e47402-8933...,Ja hallo schönen guten Tag herzlich willkommen...,2496,False,False,False,False,False,False,NaN,0,NaN
6118,Anja Haye-Heinemann,FA163E153AE3-1b27-efeac700-85631-64e4742c-7691...,Schönen guten tag Reiner Mohammad von O Kunden...,2497,False,False,False,False,False,False,NaN,0,NaN
6119,Anja Haye-Heinemann,FA163E153AE3-1b27-efeac700-85655-64e47444-e02f...,Herzlich kommen wir mal ausgeseien Service Tel...,2498,False,False,False,False,False,False,NaN,0,NaN
6120,Anja Haye-Heinemann,FA163E153AE3-1b27-efeac700-85658-64e47447-7a4a...,O Kundenservice mein Name was darf ich für Sie...,2499,False,False,False,False,False,False,NaN,0,NaN


In [15]:


filtered_df_1 = df[(df['I@H GA'] == True)]

filtered_df_1


,AP,Ariadne_internal_ID,content,cnt,VVL Mobile,VVL DSL,PO GA,I@H GA,TV GA,OOC (out of category),Comment,Conditional Formatting,Bearbeitungszeit
20,Katrin Kühne,FA163E153AE3-1b27-eeeaa700-77e49-64e30a96-59ac...,Herzlich willkommen bei O free Meister Winter ...,80,False,False,False,True,False,False,NaN,1,NaN
80,Katrin Kühne,FA163E153AE3-1b27-eeeaa700-7812b-64e30d04-57bc...,Guten Morgen Herr willkommen bei O ich heiße R...,140,False,False,True,True,False,False,NaN,2,NaN
94,Katrin Kühne,FA163E153AE3-1b27-eeeaa700-781dd-64e30d53-257a...,Hallo willkommen bei ihrem O Kundenservice war...,154,False,False,False,True,False,False,NaN,1,NaN
150,Katrin Kühne,FA163E153AE3-1b27-eeeaa700-784f3-64e31102-2abe...,Herzlich willkommen bei O Service meine Name w...,210,False,False,False,True,False,False,NaN,1,NaN
171,Katrin Kühne,FA163E153AE3-1b27-eeeaa700-78729-64e31697-d742...,Einen wunderschönen guten Tag und herzlich wil...,231,False,False,True,True,False,False,NaN,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5566,Anja Haye-Heinemann,FA163E153AE3-1b27-ef6ab700-810db-64e720a3-b42e...,Herzlich willkommen in der O Bestellhotline me...,1945,False,False,False,True,True,False,NaN,2,NaN
5949,Anja Haye-Heinemann,FA163E153AE3-1b27-efeac700-84881-64e45a1f-33b.txt,Kundenservice das ist ja das ist mein Name was...,2328,False,False,False,True,False,False,NaN,1,NaN
5994,Anja Haye-Heinemann,FA163E153AE3-1b27-efeac700-84a76-64e45bbb-332d...,Herzlich willkommen bei O sie sprechen mit die...,2373,False,False,True,True,False,False,NaN,2,NaN
6086,Anja Haye-Heinemann,FA163E153AE3-1b27-efeac700-8516e-64e46ac6-3574...,Einen wunderschönen guten Tag und herzlich wil...,2465,False,False,False,True,False,False,NaN,1,NaN


In [17]:

filtered_df_2 = df[(df['VVL DSL'] == True)]

filtered_df_2

,AP,Ariadne_internal_ID,content,cnt,VVL Mobile,VVL DSL,PO GA,I@H GA,TV GA,OOC (out of category),Comment,Conditional Formatting,Bearbeitungszeit
11,Katrin Kühne,FA163E153AE3-1b27-eeeaa700-77d89-64e307e3-81bb...,Herzlich willkommen bei O sie sprechen mit And...,71,True,True,False,False,False,False,NaN,2,NaN
14,Katrin Kühne,FA163E153AE3-1b27-eeeaa700-77db7-64e3080f-5e48...,Herzlich willkommen bei O Service ich bin hier...,74,False,True,False,False,False,False,NaN,1,NaN
39,Katrin Kühne,FA163E153AE3-1b27-eeeaa700-77f0f-64e30b27-9c7b...,Ursula Service Oliver am Apparat wie können wi...,99,False,True,False,False,False,False,NaN,1,NaN
70,Katrin Kühne,FA163E153AE3-1b27-eeeaa700-780bc-64e30cd4-6d2d...,Herzlich willkommen bei ihrem O Service sie sp...,130,False,True,False,False,False,False,NaN,1,NaN
106,Katrin Kühne,FA163E153AE3-1b27-eeeaa700-7827a-64e30d97-3e46...,Herzlich willkommen bei ihrem Auto Service sie...,166,False,True,False,False,False,False,NaN,1,NaN
639,Anja Haye-Heinemann,FA163E153AE3-1b27-efeac700-8e1c0-64e7b42f-2626...,Bekommen im ausgezeichnet Service Team von O s...,2961,False,True,False,False,False,False,NaN,1,NaN
1582,Anja Haye-Heinemann,FA163E153AE3-1b27-efeac700-8e1c0-64e7b42f-2626...,Bekommen im ausgezeichnet Service Team von O s...,2961,False,True,False,False,False,False,NaN,1,NaN
2210,Anja Haye-Heinemann,FA163E153AE3-1b27-f06ad700-7f7b4-64e72d3f-a94e...,Schönen guten Tag Herr ist der Udo Kundenservi...,3589,False,True,False,False,False,False,NaN,1,NaN
2893,Anja Haye-Heinemann,FA163E153AE3-1b27-f0eae700-83c34-64e75f23-46f6...,Guten Tag willkommen bei O sie sprechen mit Wo...,4272,False,True,False,False,False,False,NaN,1,NaN
3673,#1,FA163E153AE3-1b27-eeeaa700-77a45-64e303ad-aed2...,Hallo schönen guten Tag mein Name ist gibt es ...,35,False,True,True,False,False,False,NaN,2,02:46:00


In [18]:
vectorizer = CountVectorizer()
dtm = vectorizer.fit_transform(filtered_df_1['content'])
# Apply LDA for topic modeling
num_topics = 7  # Adjust the number of topics as needed
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(dtm)

# Get the top words for each topic
def get_top_words(model, vectorizer, n_words=10):
    words = vectorizer.get_feature_names_out()
    topics = {}
    for topic_idx, topic in enumerate(model.components_):
        top_word_idx = topic.argsort()[:-n_words - 1:-1]
        top_words = [words[i] for i in top_word_idx]
        topics[f"Topic {topic_idx + 1}"] = top_words
    return topics

topics = get_top_words(lda, vectorizer)
print("Top words for each topic:")
for topic, top_words in topics.items():
    print(f"{topic}: {', '.join(top_words)}")

Top words for each topic:
Topic 1: bewertungne, werner, festnetzvertrag, spiel, spät, drittanbieter, ele, motor, marcel, abzuschließen
Topic 2: bewertungne, werner, festnetzvertrag, spiel, spät, drittanbieter, ele, motor, marcel, abzuschließen
Topic 3: sie, das, ich, die, dann, ja, und, ist, haben, noch
Topic 4: sie, ich, das, die, und, ist, ja, der, nicht, wir
Topic 5: bewertungne, werner, festnetzvertrag, spiel, spät, drittanbieter, ele, motor, marcel, abzuschließen
Topic 6: bewertungne, werner, festnetzvertrag, spiel, spät, drittanbieter, ele, motor, marcel, abzuschließen
Topic 7: sie, und, ich, die, ein, von, bekommen, moment, das, haben


In [19]:
vectorizer = CountVectorizer()
dtm = vectorizer.fit_transform(filtered_df_2['content'])
# Apply LDA for topic modeling
num_topics = 7  # Adjust the number of topics as needed
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(dtm)

# Get the top words for each topic
def get_top_words(model, vectorizer, n_words=10):
    words = vectorizer.get_feature_names_out()
    topics = {}
    for topic_idx, topic in enumerate(model.components_):
        top_word_idx = topic.argsort()[:-n_words - 1:-1]
        top_words = [words[i] for i in top_word_idx]
        topics[f"Topic {topic_idx + 1}"] = top_words
    return topics

topics = get_top_words(lda, vectorizer)
print("Top words for each topic:")
for topic, top_words in topics.items():
    print(f"{topic}: {', '.join(top_words)}")

Top words for each topic:
Topic 1: sie, das, ich, die, dann, und, ist, ja, also, so
Topic 2: sie, das, ich, die, ja, dann, ist, und, nur, für
Topic 3: das, also, ja, sie, da, mal, ist, der, dann, ich
Topic 4: lautet, verstanden, gleiche, udo, überprüfen, kundenservice, bestätigung, besten, stellige, gebe
Topic 5: lautet, verstanden, gleiche, udo, überprüfen, kundenservice, bestätigung, besten, stellige, gebe
Topic 6: sie, für, haben, ich, und, bekommen, das, so, eine, den
Topic 7: sie, die, ich, das, haben, und, für, ist, ja, dann


In [22]:
new_df_1 = df4[(df4['I@H GA'] == 1)]
new_df_2 = df4[(df4['VVL DSL'] == 1)]



In [28]:
vectorizer = CountVectorizer()
dtm = vectorizer.fit_transform(new_df_1['text'])
# Apply LDA for topic modeling
num_topics = 10  # Adjust the number of topics as needed
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(dtm)

# Get the top words for each topic
def get_top_words(model, vectorizer, n_words=10):
    words = vectorizer.get_feature_names_out()
    topics = {}
    for topic_idx, topic in enumerate(model.components_):
        top_word_idx = topic.argsort()[:-n_words - 1:-1]
        top_words = [words[i] for i in top_word_idx]
        topics[f"Topic {topic_idx + 1}"] = top_words
    return topics

topics = get_top_words(lda, vectorizer)
print("Top words for each topic:")
for topic, top_words in topics.items():
    print(f"{topic}: {', '.join(top_words)}")

Top words for each topic:
Topic 1: sie, die, das, rufnummer, haben, bitte, ich, dann, ist, ja
Topic 2: sie, das, die, ich, dann, ja, ist, haben, für, der
Topic 3: das, sie, ja, dann, die, ich, also, hier, wir, auch
Topic 4: sie, das, was, haben, die, mail, uns, bei, ist, ich
Topic 5: sie, ich, ja, das, die, und, dann, ist, haben, für
Topic 6: dadrum, sacherschuss, jede, neukunden, nachher, ersparnis, vertragslaufzeit, mitgeben, erkennen, gemeinsames
Topic 7: schon, haben, den, gb, ihre, vertrag, kriegen, uns, gerne, sie
Topic 8: dadrum, sacherschuss, jede, neukunden, nachher, ersparnis, vertragslaufzeit, mitgeben, erkennen, gemeinsames
Topic 9: dadrum, sacherschuss, jede, neukunden, nachher, ersparnis, vertragslaufzeit, mitgeben, erkennen, gemeinsames
Topic 10: dadrum, sacherschuss, jede, neukunden, nachher, ersparnis, vertragslaufzeit, mitgeben, erkennen, gemeinsames


In [27]:
vectorizer = CountVectorizer()
dtm = vectorizer.fit_transform(new_df_2['text'])
# Apply LDA for topic modeling
num_topics = 7  # Adjust the number of topics as needed
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(dtm)

# Get the top words for each topic
def get_top_words(model, vectorizer, n_words=10):
    words = vectorizer.get_feature_names_out()
    topics = {}
    for topic_idx, topic in enumerate(model.components_):
        top_word_idx = topic.argsort()[:-n_words - 1:-1]
        top_words = [words[i] for i in top_word_idx]
        topics[f"Topic {topic_idx + 1}"] = top_words
    return topics

topics = get_top_words(lda, vectorizer)
print("Top words for each topic:")
for topic, top_words in topics.items():
    print(f"{topic}: {', '.join(top_words)}")

Top words for each topic:
Topic 1: sie, das, die, ich, den, für, und, wir, dann, ist
Topic 2: los, aufgabe, gefreut, liebste, ern, emil, liebsten, allerdings, surfen, werktagen
Topic 3: los, aufgabe, gefreut, liebste, ern, emil, liebsten, allerdings, surfen, werktagen
Topic 4: los, aufgabe, gefreut, liebste, ern, emil, liebsten, allerdings, surfen, werktagen
Topic 5: sie, ja, ich, die, das, ist, dann, und, der, genau
Topic 6: los, aufgabe, gefreut, liebste, ern, emil, liebsten, allerdings, surfen, werktagen
Topic 7: sie, das, ich, ja, die, und, dann, jetzt, ist, noch
